In [11]:
import pandas as pd
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [12]:
enem = pd.read_csv("train.csv")
enem_teste = pd.read_csv("test.csv")

dropped_columns = []
for col in enem:
    if col != 'IN_TREINEIRO' and ((enem[col].dtypes != "float64" and enem[col].dtypes != "int64") or (col.startswith("Q0") ) or col not in enem_teste.columns):
        dropped_columns.append(col)
enem = enem.drop(columns=dropped_columns)

In [13]:
dropped_columns = []
for col in enem_teste:
    if col not in enem.columns:
        dropped_columns.append(col)
enem_teste = enem_teste.drop(columns=dropped_columns)

In [14]:
enem_teste = enem_teste.fillna(0)
enem = enem.fillna(0)

In [15]:
enem.groupby(['IN_TREINEIRO']).count()

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,...,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
IN_TREINEIRO,,,,,,,,,,,,,,,,,,,,,
0,11947,11947,11947,11947,11947,11947,11947,11947,11947,11947,...,11947,11947,11947,11947,11947,11947,11947,11947,11947,11947
1,1783,1783,1783,1783,1783,1783,1783,1783,1783,1783,...,1783,1783,1783,1783,1783,1783,1783,1783,1783,1783


In [16]:
df_nao_treineiro = enem[enem.IN_TREINEIRO==0]
df_treineiro = enem[enem.IN_TREINEIRO==1]

In [17]:
df_treineiro_upsampled = resample(df_treineiro, 
                                 replace=True,     # sample with replacement
                                 n_samples=11947,    # to match majority class
                                 random_state=1245) # reproducible results
enem_upsampled = pd.concat([df_nao_treineiro, df_treineiro_upsampled])
enem_upsampled.IN_TREINEIRO.value_counts()

1    11947
0    11947
Name: IN_TREINEIRO, dtype: int64

In [18]:
treineiro = enem_upsampled.IN_TREINEIRO
X = enem_upsampled.drop('IN_TREINEIRO', axis=1)
logistic_regression = LogisticRegression().fit(X, treineiro)
random_forest_classifier = RandomForestClassifier().fit(X, treineiro)
ada_forest_classifier = AdaBoostClassifier().fit(X, treineiro)

In [19]:
pred_logistic_regression = logistic_regression.predict(X)
pred_random_forest_classifier = random_forest_classifier.predict(X)
pred_ada_classifier = ada_forest_classifier.predict(X)
print( accuracy_score(pred_logistic_regression, treineiro) )
print( accuracy_score(pred_random_forest_classifier, treineiro) )
print( accuracy_score(pred_ada_classifier, treineiro) )

0.9661421277308111
0.9993303758265674
0.9971959487737507


In [20]:
prediction = random_forest_classifier.predict(enem_teste)

In [21]:
test = pd.read_csv("test.csv")
result = pd.DataFrame()

In [22]:
result['NU_INSCRICAO'] = test['NU_INSCRICAO']
result['IN_TREINEIRO'] = prediction

In [23]:
result.to_csv('answer.csv', index=False)